In [1]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os
import joblib

In [2]:
# Function to extract MFCC features from audio
def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=3.0)  # Load 3 seconds
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # 13 MFCCs
    return np.mean(mfccs, axis=1)  # Average over time

# Load dataset
data_dir = "data/"
instruments = ["Guitar", "Piano", "Drums", "Violin", "Flute"]
X, y = [], []

In [ ]:
print("Loading dataset...")
for instrument in instruments:
    folder = f"{data_dir}/{instrument}"
    if not os.path.exists(folder):
        print(f"Warning: {folder} not found. Skipping.")
        continue
    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = f"{folder}/{file}"
            try:
                features = extract_features(file_path)
                X.append(features)
                y.append(instrument)
                print(f"Loaded: {file_path}")
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

X = np.array(X)
y = np.array(y)

if len(X) == 0:
    raise ValueError("No audio files loaded. Check your data folder.")

In [4]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [ ]:
# Train model
print("Training model...")
model = SVC(kernel='linear', probability=True)  # Linear SVM with probabilities
model.fit(X_train, y_train)

In [ ]:
from sklearn.neural_network import MLPClassifier
model2 = MLPClassifier(hidden_layer_sizes=(80, 30), max_iter=500, random_state=42)
model2.fit(X_train, y_train)

In [38]:
# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy1: {accuracy * 100:.2f}%")

Test Accuracy1: 82.61%


In [44]:
# Evaluate
y_pred = model2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy2: {accuracy * 100:.2f}%")

Test Accuracy2: 91.30%


In [ ]:
# Save models
joblib.dump(model, "instrument_model.pkl")
print("Model saved as instrument_model.pkl")
joblib.dump(model2, "instrument_model2.pkl")
print("Model saved as instrument_model2.pkl")

In [48]:
# Prediction function
def predict_instrument(file_path):
    features = extract_features(file_path)
    pred = model.predict([features])[0] # use model if you want SVC and use model2 if you want MLP
    probs = model.predict_proba([features])[0] # use model if you want SVC and use model2 if you want MLP
    confidence = max(probs)
    return pred, confidence

In [52]:
# Test prediction
test_file = "data/tests/test10.wav"  # Replace with a real file
if os.path.exists(test_file):
    instrument, confidence = predict_instrument(test_file)
    print(f"Predicted: {instrument} (Confidence: {confidence:.2f})")
else:
    print(f"Test file {test_file} not found. Add a sample to test.")

Predicted: Piano (Confidence: 0.74)
